**初期ディレクトリ構成**
```
preprocess.ipynb
train.ipynb
images/
  - 0.white/
    - 850/
      - PP_*.(png|jpg)
      - PE_*.(png|jpg)
      - PET_*.(png|jpg)
      - PS_*.(png|jpg)
    - 940/
  - 1.gray/
  - 2.black/
  - 3.red/
  - 4.brown/
  - 5.yellow/
  - 6.green/
  - 7.blue/
```

**画像サイズ**
- 128×128

**前提**
- 850nmと940nmの画像数が等しい

**入力 <np.ndarray (513,)>**
- 0番目: プラスチックの色を表す番号 <np.int32>
- 1番目: 850nm 輝度0のピクセル数 <np.int32>
- 2番目: 850nm 輝度1のピクセル数 <np.int32>
- …
- 256番目: 850nm 輝度255のピクセル数 <np.int32>
- 257番目: 940nm 輝度0のピクセル数 <np.int32>
- 258番目: 940nm 輝度1のピクセル数 <np.int32>
- …
- 512番目: 940nm 輝度255のピクセル数 <np.int32>

**出力 <np.int32>**
- プラスチックの材質を表す番号

**プラスチックの色を表す番号**
0. white
1. gray
2. black
3. red
4. brown
5. yellow
6. green
7. blue

**プラスチックの材質を表す番号**
0. PP
1. PE
2. PET
3. PS

**CSV (data.csv)**
- plastic_type: プラスチックの材質を表す番号
- plastic_color: プラスチックの色を表す番号
- led850_0: 850nm 輝度0のピクセル数
- led850_1: 850nm 輝度1のピクセル数
- …
- led850_255: 850nm 輝度255のピクセル数
- led940_0: 940nm 輝度0のピクセル数
- led940_1: 940nm 輝度1のピクセル数
- …
- led940_255: 940nm 輝度255のピクセル数

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/trashart

/content/drive/MyDrive/trashart


In [3]:
import numpy as np
import pandas as pd
import cv2
from PIL import Image

import os
from glob import glob

In [4]:
plastic_types = ["PP", "PE", "PET", "PS"]
plastic_types_map = {
  "PP": 0,
  "PE": 1,
  "PET": 2,
  "PS": 3
}

In [5]:
def generate_columns() -> list:
  lis = [""] * (2 + 256 * 2)

  lis[0] = "plastic_type"
  lis[1] = "plastic_color"

  for i, led in enumerate(["850", "940"]):
    for j in range(256):
      lis[2 + 256 * i + j] = f"led{led}_{j}"
  
  return lis

In [6]:
class Preprocesser:
  def __init__(self, p_type: str, p_no: int, color_no: int, dir_name: str):
    self.p_type = p_type
    self.p_no = p_no
    self.color_no = color_no
    self.dir_name = dir_name
    self.led850_paths = sorted(glob(f"./images/{dir_name}/850/{p_type}_*", recursive=True))
    self.led940_paths = sorted(glob(f"./images/{dir_name}/940/{p_type}_*", recursive=True))
    self.led850_imgs = self._load_images(self.led850_paths)
    self.led940_imgs = self._load_images(self.led940_paths)
    self.img_num = len(self.led850_paths)

    self.info = np.zeros((self.img_num, 2), dtype=np.int32)
    self.luminances = np.zeros((self.img_num, 256*2), dtype=np.int32)

    print(f"<< 基本情報の挿入 << {self.dir_name} {self.p_type} - 開始")
    self._insert_info()
    print(f"<< 基本情報の挿入 << {self.dir_name} {self.p_type} - 完了")

    print(f"<< 輝度情報の挿入 << {self.dir_name} {self.p_type} - 開始")
    self._insert_luminance()
    print(f"<< 輝度情報の挿入 << {self.dir_name} {self.p_type} - 終了")

  def _load_images(self, paths: list) -> list:
    print(f">> 画像の読み込み >> {self.dir_name} {self.p_type} - 開始")
    imgs = [None] * len(paths)

    for i, path in enumerate(paths):
      pil_img = Image.open(path)
      imgs[i] = np.array(pil_img)

    print(f">> 画像の読み込み >> {self.dir_name} {self.p_type} - 完了 ({len(paths)}枚)")
    return imgs
  
  def _insert_info(self):
    for i, path in enumerate(self.led850_paths):
      self.info[i, 0] = self.p_no
      self.info[i, 1] = self.color_no
  
  def _insert_luminance(self):
    for i in range(self.img_num):
      self.luminances[i] = self._calc_luminance(i)

  def _calc_luminance(self, index: int) -> np.ndarray:
    hls = cv2.cvtColor(self.led850_imgs[index], cv2.COLOR_BGR2HLS)
    h,l,s = cv2.split(hls)
    raveled = l.ravel()

    led850_count = np.zeros(256, dtype=np.int32)
    for i in range(256):
        led850_count[i] = raveled[raveled == i].shape[0]
    
    hls = cv2.cvtColor(self.led940_imgs[index], cv2.COLOR_BGR2HLS)
    h,l,s = cv2.split(hls)
    raveled = l.ravel()

    led940_count = np.zeros(256, dtype=np.int32)
    for i in range(256):
        led940_count[i] = raveled[raveled == i].shape[0]
    
    return np.concatenate([led850_count, led940_count])
  
  def result(self) -> np.ndarray:
    return np.concatenate([self.info, self.luminances], axis=1)

In [7]:
colors = sorted(os.listdir("./images"))
colors

['0.white',
 '1.gray',
 '2.black',
 '3.red',
 '4.brown',
 '5.yellow',
 '6.green',
 '7.blue']

In [8]:
data = np.zeros((0, 2+256*2), dtype=np.int32)

for color_no, dir_name in enumerate(colors):
  for p_type in plastic_types:
    ins = Preprocesser(p_type, plastic_types_map[p_type], color_no, dir_name)
    data = np.concatenate([data, ins.result()])
  print()

>> 画像の読み込み >> 0.white PP - 開始
>> 画像の読み込み >> 0.white PP - 完了 (15枚)
>> 画像の読み込み >> 0.white PP - 開始
>> 画像の読み込み >> 0.white PP - 完了 (15枚)
<< 基本情報の挿入 << 0.white PP - 開始
<< 基本情報の挿入 << 0.white PP - 完了
<< 輝度情報の挿入 << 0.white PP - 開始
<< 輝度情報の挿入 << 0.white PP - 終了
>> 画像の読み込み >> 0.white PE - 開始
>> 画像の読み込み >> 0.white PE - 完了 (15枚)
>> 画像の読み込み >> 0.white PE - 開始
>> 画像の読み込み >> 0.white PE - 完了 (15枚)
<< 基本情報の挿入 << 0.white PE - 開始
<< 基本情報の挿入 << 0.white PE - 完了
<< 輝度情報の挿入 << 0.white PE - 開始
<< 輝度情報の挿入 << 0.white PE - 終了
>> 画像の読み込み >> 0.white PET - 開始
>> 画像の読み込み >> 0.white PET - 完了 (2枚)
>> 画像の読み込み >> 0.white PET - 開始
>> 画像の読み込み >> 0.white PET - 完了 (2枚)
<< 基本情報の挿入 << 0.white PET - 開始
<< 基本情報の挿入 << 0.white PET - 完了
<< 輝度情報の挿入 << 0.white PET - 開始
<< 輝度情報の挿入 << 0.white PET - 終了
>> 画像の読み込み >> 0.white PS - 開始
>> 画像の読み込み >> 0.white PS - 完了 (0枚)
>> 画像の読み込み >> 0.white PS - 開始
>> 画像の読み込み >> 0.white PS - 完了 (0枚)
<< 基本情報の挿入 << 0.white PS - 開始
<< 基本情報の挿入 << 0.white PS - 完了
<< 輝度情報の挿入 << 0.white PS - 開始
<< 輝度情報の挿入 << 0.wh

In [9]:
df = pd.DataFrame(data=data, columns=generate_columns())
df

,plastic_type,plastic_color,led850_0,led850_1,led850_2,led850_3,led850_4,led850_5,led850_6,led850_7,...,led940_246,led940_247,led940_248,led940_249,led940_250,led940_251,led940_252,led940_253,led940_254,led940_255
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,1,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90,1,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,1,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92,1,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df.to_csv("data.csv", index=False)